# Bleu score:
## Définition:
BLEU (the Bilingual Evaluation Understudy), est un algorithme d'évaluation automatique d'une MT (machine traduction) qui a été proposé par Kishore Papineni, Salim Roukos, Todd Ward, and Wei-Jing Zhu à 2002, cet algorithme compare une traduction candidate avec une traduction référence et calcule un score entre 0 et 1.

## Approche théorique:
\begin{equation*}
BLEU = BP\prod_{k=1}^{n}{p_k}^{w_k}=e^{ln(BP)}e^{\sum^{n}_{k=1}w_kln(p_k)}
\end{equation*}
$
Avec: BP = min(1, \frac{hypothesis\ lenght}{reference\ lenght})\quad et \quad p_k = \frac{correct\ k-grams}{total\ k-grams}
$


L'idée principale de BLEU est de comparer les n-grams d'une traduction candidate avec les n-grams d'une ou plusieurs traductions références et compter le nombre de sémelarité (calculer une précision pour chaque k-gram 0<k<n).
cette méthode respecte les 2 aspects de traduction:
- adequacy: Combien de la signification exprimée dans la traduction standard (référence) ou la source est également exprimée dans la traduction candidate. et donc une traduction candidate qui a les mêmes mots (1-grams) d'une traduction référence servi à respecter cette norme.
- fluency: est-ce qu'une traduction candidate est grammaticalement bien informée ou non. ce qui peut être mesuré par les n-grams.

L'algorithme BLEU prit en considération aussi qu'une traduction candidate ne doit pas être trop longue ni trop court par rapport au référence. donc:
- si elle est trop longue que sa référence, le score calculé va être pénalisé par les n-grams.
- si elle est trop court que sa référence, le score calculé va être pénalisé par un facteur multiplicative (BP).
donc une bonne traduction doit avoir une longueur, un ordre des mots, un choix des mots proches de sa référence.

## Approche pratique:

Natural Language Toolkit library (NLTK) propose une implémentation de BLEU qu'on va utiliser dans notre projet.

### Initialisation:

In [2]:
reference = ["my name is hamza jamal".split()]
candidate = "hamza jamal is a name of me".split()

### Précision:
#### Les n-grams:

In [2]:
from nltk import ngrams

precisions = []
print("candidate n-grams:")
for i in range(1,5):
    cand_i_grams = list(ngrams(candidate, i))
    print("{}-grams: {}".format(i,cand_i_grams))
print()
print("reference n-grams:")
for i in range(1,5):
    ref_i_grams = list(ngrams(*reference, i))
    print("{}-grams: {}".format(i,ref_i_grams))

candidate n-grams:
1-grams: [('hamza',), ('jamal',), ('is',), ('a',), ('name',), ('of',), ('me',)]
2-grams: [('hamza', 'jamal'), ('jamal', 'is'), ('is', 'a'), ('a', 'name'), ('name', 'of'), ('of', 'me')]
3-grams: [('hamza', 'jamal', 'is'), ('jamal', 'is', 'a'), ('is', 'a', 'name'), ('a', 'name', 'of'), ('name', 'of', 'me')]
4-grams: [('hamza', 'jamal', 'is', 'a'), ('jamal', 'is', 'a', 'name'), ('is', 'a', 'name', 'of'), ('a', 'name', 'of', 'me')]

reference n-grams:
1-grams: [('my',), ('name',), ('is',), ('hamza',), ('jamal',)]
2-grams: [('my', 'name'), ('name', 'is'), ('is', 'hamza'), ('hamza', 'jamal')]
3-grams: [('my', 'name', 'is'), ('name', 'is', 'hamza'), ('is', 'hamza', 'jamal')]
4-grams: [('my', 'name', 'is', 'hamza'), ('name', 'is', 'hamza', 'jamal')]


### Calculer la précision:

In [3]:
from nltk.translate.bleu_score import modified_precision

precisions = []
for i in range(1,5):
    precision = modified_precision(reference, candidate, i)
    precisions.append(precision)
    print("précision de {0}-gram = {1}".format(i,precision))

précision de 1-gram = 4/7
précision de 2-gram = 1/6
précision de 3-gram = 0/5
précision de 4-gram = 0/4


### Facteur de pénalisation:

In [4]:
from nltk.translate.bleu_score import brevity_penalty

ref_len = len(reference)
cand_len = len(candidate)
BP = brevity_penalty(ref_len, cand_len)
print(BP)

1


### Calculer Bleu score:

In [5]:
from math import exp, log
# exp(log(0))
exp_log_precisions = map(lambda p_k:exp(1/4*log(p_k)), precisions)
BLEU_score=BP
for exp_log_precision in exp_log_precisions:
    BLEU_score *= exp_log_precision
print(BLEU_score)

ValueError: math domain error

### Exemples:
#### - 1 phrase 1 référence:

In [3]:
from nltk.translate.bleu_score import sentence_bleu

# score = sentence_bleu(reference, candidate)
## solutions:
# 1- préciser le n:
# score = sentence_bleu(reference, candidate, weights=(0.5,0.5))
# 2- utiliser smoothing functions:
# from nltk.translate.bleu_score import SmoothingFunction
# my_smoothing_function = SmoothingFunction()
# score = sentence_bleu(reference, candidate, smoothing_function=my_smoothing_function.method0)
# 3- utiliser emulate_multibleu
# score = sentence_bleu(reference, candidate, emulate_multibleu=True)

print(score)

0.0


#### - 1 phrase plusieurs références:

In [7]:
from nltk.translate.bleu_score import sentence_bleu
references = ["my name is hamza jamal how about you?".split(), "hamza jamal is my name and you?".split()]
candidate_2 = "hamza jamal is my name how about you?".split()
score = sentence_bleu(references, candidate_2)
print(score)

0.691441569283882


#### - text (plusieurs phrases) plusieurs références:

In [ ]:
pass